<a href="https://colab.research.google.com/github/andrija-sretovic/Fanatsy-PL-analysis-/blob/main/Fantasy_PL_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fantasy Premier League Analysis

Let's use this break to take a look at some statistical data frmo the PL sesaon so far. New wildcard is available some players might be thinking about using it, especially with so many injuries and players going away for AFCON and Asian Cup. This is purely statistical, so it might not match the Fantsy points exactly, but it should show some guidelines in what direction the season is going.

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
shooting_df = pd.read_csv("['shooting'].csv")
passing_df = pd.read_csv("['passing'].csv")
gca_df = pd.read_csv("['gca'].csv")
pass_types_df = pd.read_csv("['passing', 'types'].csv")
keeper_df = pd.read_csv("['keeper'].csv")
team_defence_df = pd.read_csv('shooting against-07-01-2024')

In [5]:
# Drop unnessesary columns
shooting_df.drop(['Rk', 'Nation', 'Age', 'Born', 'FK', 'PK', 'np:G-xG', 'Matches', 'Dist', 'npxG/Sh', 'G-xG',	'SoT',	'SoT%',	'SoT/90',	'G/SoT'], axis=1, inplace=True)

# Switch numeric columns from string to numeric
shooting_df[['90s',	'Gls',	'Sh',	'Sh/90',	'G/Sh',	'PKatt',	'xG',	'npxG']] = shooting_df[['90s',	'Gls',	'Sh',	'Sh/90',	'G/Sh',	'PKatt',	'xG',	'npxG']].apply(pd.to_numeric, errors='coerce')

# Drop player column which contains player due to scraping error. Later to adjust in the scrape code
shooting_df.drop(shooting_df[shooting_df['Player'] == 'Player'].index, inplace=True)

shooting_df.head()

,Player,Pos,Squad,90s,Gls,Sh,Sh/90,G/Sh,PKatt,xG,npxG
0,Max Aarons,DF,Bournemouth,11.10,0.00,1.00,0.09,0.00,0.00,0.00,0.00
1,Bénie Adama Traore,"FW,MF",Sheffield Utd,4.30,0.00,1.00,0.23,0.00,0.00,0.30,0.30
2,Tosin Adarabioyo,DF,Fulham,6.00,1.00,3.00,0.50,0.33,0.00,0.00,0.00
3,Elijah Adebayo,FW,Luton Town,9.20,5.00,18.00,1.96,0.28,0.00,3.70,3.70
4,Simon Adingra,"FW,MF",Brighton,12.70,4.00,20.00,1.58,0.20,0.00,2.40,2.40


In [6]:
# Singling out the forwards and midfielders
shooting = shooting_df[(shooting_df.Pos == 'FW') | (shooting_df.Pos.apply(lambda x: ('FW' in x.split(',')))) | (shooting_df.Pos == 'MF') | (shooting_df.Pos.apply(lambda x: x.split(',')[0] == 'MF'))]

# Players who have played more than 50% of games.
shooting = shooting[shooting['90s'] > (int(shooting['90s'].max()) / 2)]
shooting.sample(5)

,Player,Pos,Squad,90s,Gls,Sh,Sh/90,G/Sh,PKatt,xG,npxG
199,Erling Haaland,FW,Manchester City,14.40,14.00,54.00,3.76,0.20,4.00,14.80,11.60
175,Idrissa Gana Gueye,MF,Everton,11.20,1.00,15.00,1.33,0.07,0.00,1.00,1.00
55,Philip Billing,MF,Bournemouth,13.00,2.00,23.00,1.77,0.09,0.00,2.10,2.10
317,Scott McTominay,MF,Manchester Utd,12.40,5.00,28.00,2.26,0.18,0.00,3.60,3.60
227,Vitaly Janelt,"MF,DF",Brentford,17.10,0.00,10.00,0.58,0.00,0.00,1.10,1.10


In [14]:
# Add XG per shot column, we want players that take good chances. Players who take a lot of shots can boost their XG without geting in good positions
shooting['xG/shot'] = round(shooting.xG / shooting.Sh, 2)
shooting.sort_values('xG/shot', ascending=False, inplace=True)

# Now add column which multiplies xG/shot and Shot/90 to get the xG we can expect from a pplayer each game
shooting['xG/90'] = shooting['xG/shot'] * shooting['Sh/90']

shooting.sample(5)

,Player,Pos,Squad,90s,Gls,Sh,Sh/90,G/Sh,PKatt,xG,npxG,xG/shot,Surname,xG/90
224,Alex Iwobi,"FW,MF",Fulham,12.30,3.00,32.00,2.60,0.09,0.00,3.40,3.40,0.11,Iwobi,0.29
463,Dominik Szoboszlai,MF,Liverpool,17.70,2.00,38.00,2.15,0.05,0.00,1.80,1.80,0.05,Szoboszlai,0.11
317,Scott McTominay,MF,Manchester Utd,12.40,5.00,28.00,2.26,0.18,0.00,3.60,3.60,0.13,McTominay,0.29
468,Marcus Tavernier,FW,Bournemouth,13.00,2.00,31.00,2.38,0.06,0.00,2.70,2.70,0.09,Tavernier,0.21
380,Cole Palmer,"FW,MF",Chelsea,12.70,8.00,34.00,2.68,0.12,4.00,7.00,3.90,0.21,Palmer,0.56


In [8]:
# Filter out players which take more shots than two shots per game
shooting = shooting[shooting['Sh/90'] > 2]

# Add surname column to make it easier to read when ploting
shooting['Surname'] = shooting.Player.apply(lambda x: x.split(' ')[1] if len(x.split(' ')) > 1 else x)

<ipython-input-8-650fefde11bd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shooting['Surname'] = shooting.Player.apply(lambda x: x.split(' ')[1] if len(x.split(' ')) > 1 else x)


# Let's take a look at the amount of shots players take and how good the average quality of the chance is

In [22]:
# Single out the best 20 players with xG/90
xg90 = shooting[['Surname', 'xG/90']].sort_values('xG/90').tail(20)

In [21]:
fig = px.bar(xg90,
             x='xG/90',
             y='Surname',
             color='xG/90'
             )

fig.show()

Haaland has an amazing stat that he should be scoring a goal every 90 minutes he spends on the pitch. Which is not far from reality (14 goals in 14.4 90s). Isak and Nunez are interesting. They create a lot but we need to analyze them further to see the reality. Rocharlison seems to be picking up his form and could be an interesting option. And the price of Cole Palmer makes him a must-have in my opinion.

Let's take a look into playing time a little further.

In [27]:
fig = px.scatter(shooting,
                 x='90s',
                 y='xG/90',
                 size='Sh/90',
                 text='Surname',
                 title= "Ploting number of games played taken against xG per 90",
                 color='xG/shot'
                 )

fig.update_layout(xaxis_title='Number of 90s player has played',
                  yaxis_title='xG per game'
                  )

fig.show()

Here we can see that Nunez and Isaak are simlar in terms of the expected output, just the big difference is the that Nunez takes almost double the shots. Either of them can be an interesting option, but only if Isaak can stay fit or if Nunez starts scoring some of those chances.

Salah and Solanke stand out in terms of playing regularly and getting into good chances. Watkins steps ahead of Bowen, Son and Saka. But we still haven't gotten to the creative side of things.

Richarlison needs to play more to be considered properly. Joao Pedro can be a good cheap third striker option.

Jackson's numbers look really good, but he is a mystery and costs too much to be considered seriously. Eye test shows that there are at better options. Maybe a H2H difference maker.

In [28]:
# Load and clean passing data
passing_df = pd.read_csv("['passing'].csv")

In [29]:
# Drop unnessesary passing data columns
passing_df.drop(['Rk', 'Nation', 'Age', 'Cmp', 'Att', 'A-xAG', 'PrgP', 'Matches', 'Born', 'Cmp%', 'TotDist', 'PrgDist', 'Cmp.1', 'Att.1', 'Cmp.2', 'Att.2', 'Cmp%.1', 'Cmp%.2', 'Cmp%.3', 'Cmp.3', 'Att.3', 'xAG', '1/3'], axis=1, inplace=True)

# Switch numeric columns from string to numeric
passing_df[['90s',	'Ast',	'xA',	'KP',	'PPA',	'CrsPA']] = passing_df[['90s',	'Ast',	'xA',	'KP',	'PPA',	'CrsPA']].apply(pd.to_numeric, errors='coerce')

# Drop player column which contains player due to scraping error. Later to adjust in the scrape code
passing_df.drop(passing_df[passing_df['Player'] == 'Player'].index, inplace=True)

passing_df.sample(5)

,Player,Pos,Squad,90s,Ast,xA,KP,PPA,CrsPA
486,Leandro Trossard,"FW,MF",Arsenal,7.40,1.00,1.30,13.00,15.00,3.00
459,Jason Steele,GK,Brighton,11.00,0.00,0.00,0.00,0.00,0.00
507,James Ward-Prowse,MF,West Ham,18.70,6.00,4.00,36.00,11.00,3.00
370,Frank Onyeka,MF,Brentford,8.20,1.00,1.20,8.00,7.00,3.00
446,Marcos Senesi,DF,Bournemouth,13.40,2.00,1.30,9.00,15.00,6.00


In [30]:
# Drop pass types columns
pass_types_df.drop(['Rk', 'Nation', 'Age', 'Born', 'Att', 'Live', 'Dead', 'Sw', 'TB', 'Crs', 'TI', 'In', 'Out', 'Str', 'Cmp', 'Off', 'Blocks', 'Matches'], axis=1, inplace=True)

# Switch numeric columns from string to numeric
pass_types_df[['90s',	'FK',	'CK']] = pass_types_df[['90s',	'FK',	'CK']].apply(pd.to_numeric, errors='coerce')

# Drop player column which contains player due to scraping error. Later to adjust in the scrape code
pass_types_df.drop(pass_types_df[pass_types_df['Player'] == 'Player'].index, inplace=True)

In [32]:
# Merge the two DataFrames|
creativity_df = passing_df.merge(pass_types_df, on=['Player', 'Pos', 'Squad', '90s'])

In [34]:
# Filter out the midfielders and attackers
creativity = creativity_df[(creativity_df.Pos == 'FW') | (creativity_df.Pos.apply(lambda x: ('FW' in x.split(',')))) | (creativity_df.Pos == 'MF') | (creativity_df.Pos.apply(lambda x: x.split(',')[0] == 'MF'))]

# Players who have played more than 50% of games.
creativity = creativity[creativity['90s'] > (int(creativity['90s'].max()) / 2)]

In [35]:
# Add column for dead balls (Free kicks and corners)
creativity['DB'] = creativity.FK + creativity.CK

creativity.sort_values('DB', ascending=False, inplace=True)

# Add surname column to make it easier to read when ploting
creativity['Surname'] = creativity.Player.apply(lambda x: x.split(' ')[1] if len(x.split(' ')) > 1 else x)

creativity.sample(5)

,Player,Pos,Squad,90s,Ast,xA,KP,PPA,CrsPA,FK,CK,DB,Surname
67,Josh Brownhill,MF,Burnley,16.70,0.00,1.30,17.00,8.00,0.00,23.00,40.00,63.00,Brownhill
342,Darwin Núñez,FW,Liverpool,12.40,6.00,2.20,19.00,19.00,0.00,2.00,0.00,2.00,Núñez
189,Bruno Guimarães,MF,Newcastle Utd,18.60,2.00,2.60,26.00,34.00,1.00,48.00,0.00,48.00,Guimarães
54,Yves Bissouma,MF,Tottenham,13.40,0.00,1.20,6.00,8.00,0.00,35.00,0.00,35.00,Bissouma
219,Vitaly Janelt,"MF,DF",Brentford,17.10,1.00,2.30,23.00,18.00,7.00,10.00,0.00,10.00,Janelt


## Plot DB Ball specialist against expected assist

In [36]:
fig = px.scatter(creativity,
                 x='DB',
                 y='xA',
                 size='Ast',
                 text='Surname',
                 title= "Players dead ball situations taken against expected assist",
                 color='90s'
                 )

fig.update_layout(xaxis_title='Number of deadball situation',
                  yaxis_title='xA'
                  )

fig.show()

Taking a player just based on the fact that he takes a lot of free kicks or corners is probably only as a cheap option. Here is where Douglas Luiz and Gross can be interesting. Their price is good and you can include them in your squad as an option. Probably Gross is the more likely option.

In [37]:
# Create column for passes in Penalty Area
creativity['PA'] = creativity.PPA + creativity.CrsPA

creativity.sort_values('PA', ascending=False, inplace=True)

## Plot passes into Penalty Area against KeyPasses (passes that result in a shot)

In [38]:
fig = px.scatter(creativity,
                 x='PA',
                 y='KP',
                 size='Ast',
                 text='Surname',
                 title= "Players passes into the penalty area against key passes",
                 color='90s'
                 )

fig.update_layout(xaxis_title='Number of passes into PA',
                  yaxis_title='Key Passes (passes that lead to a shot)'
                  )

fig.show()

Bruno is the clear champion here, but are you willing to put your bet on Man United? Kulesevski and Odegaard are getting so many balls into shooting positions, but their assist tally is rather low. For their price it's probably not worth the risk. Gross pops up here once again at a decent price. I guess there is some argument for Silva, Foden or Alvarez, but are you wiling to play Pep roulette?

In [39]:
# Clean the shooting column before merging
shooting.drop(['Sh/90', 'G/Sh', 'PKatt', 'npxG'], axis=1, inplace=True)

In [40]:
# Clean the creativity df further
creativity.drop(['PPA', 'CrsPA', 'CK', 'FK'], axis=1, inplace=True)

In [41]:
# connect the two dataframes
full = shooting.merge(creativity, on=['Player', 'Pos', 'Squad', '90s', 'Surname'], how='left')

full.sample(5)

,Player,Pos,Squad,90s,Gls,Sh,xG,xG/shot,Surname,xG/90,Ast,xA,KP,DB,PA
29,Marcus Rashford,FW,Manchester Utd,14.70,3.00,41.00,5.00,0.12,Rashford,0.33,2.00,2.10,14.00,2.00,21.00
2,Nicolas Jackson,FW,Chelsea,16.10,7.00,45.00,10.60,0.24,Jackson,0.67,2.00,1.70,17.00,0.00,11.00
44,Luca Koleosho,"MF,FW",Burnley,10.80,1.00,23.00,2.00,0.09,Koleosho,0.19,1.00,1.10,11.00,2.00,8.00
48,Antony,FW,Manchester Utd,10.00,0.00,24.00,1.40,0.06,Antony,0.14,0.00,1.40,16.00,5.00,21.00
37,Bruno Fernandes,"MF,FW",Manchester Utd,19.00,3.00,45.00,4.70,0.10,Fernandes,0.24,3.00,5.50,54.00,107.00,64.00


In [42]:
# Add total output column G + A
full['GA'] = full.Gls + full.Ast

# Add column for expected assist and goal
full['XCA'] = full.xG + full.xA

full.sort_values('GA', ascending=False, inplace=True)

In [48]:
# Filter out the top 20 players by Goal + Assist output
output = full[['Surname', 'GA']].sort_values('GA').tail(20)

fig = px.bar(output,
             x='GA',
             y='Surname',
             color='GA'
             )

fig.update_layout(xaxis_title='Number of Goals + Assists',
                  yaxis_title='Players'
                  )

fig.show()

## Plot actual versus expected to see who is overachieving and who we can expect more from

In [45]:
fig = px.scatter(full,
                 x='GA',
                 y='XCA',
                 size='90s',
                 text='Surname',
                 title= "Players passes into the penalty area against key passes",
                 )

fig.update_layout(xaxis_title='Goals + Assists',
                  yaxis_title='Expected goals + assists'
                  )

fig.add_shape(
    type='line',
    x0=full['GA'].min(),
    y0=full['GA'].min(),
    x1=full['GA'].max(),
    y1=full['GA'].max(),
    line=dict(color='red', width=2)
)

fig.show()

Players under the line are overachieving and those below it are underachieving and we should expect more. Saka will show even more and should be part of your squad. Jackson and Fernandes are there on the list of underachievers, but I've spoken about them. They are a risky option.I guess DCL should get some points if he manages to stay fit. But Pedro and Cunha are a better option. Or Isaak if he can stay fit and pass the ball. We can see that Elanga and Bailey are overachieving by quite a bit and we probably can expect their number to slow down a little bit.

## Plot players who get in good positions but are also good passers

In [47]:
fig = px.scatter(full,
                 x='xG/90',
                 y='xA',
                 size='GA',
                 color='90s',
                 text='Surname',
                 title= "Players passes into the penalty area against Expected Assists",
                 )

fig.update_layout(xaxis_title='Expected goals per 90',
                  yaxis_title='Expected Assists'
                  )

fig.add_shape(
    type='line',
    x0=full['xG/90'].min(),
    y0=full['xA'].min(),
    x1=full['xG/90'].max(),
    y1=full['xA'].max(),
    line=dict(color='black', width=2)
)

fig.show()

Players under the line can be considered greedy, which is fine if they are giving the end product (bigger circle), but players like Isak and DCL really need to start passing or scoring. This is a sign of bad decision making. And is not a good sign going forward.

Players above the line are creators, but also if their circle is not big. They obviously need to get into better positions. This is obvious for Fernandes and Odegaard.

Saka has so much potential to grow further. There is space for him to impace the game even further by svcoring more.

Watkins, Mbeumo and Palmer are on the sweet spot. Getting into good postions and also creating for teammates.



# Lets's turn to defence now. First let's look at teams as a whole, because what good are individual stats if the player is not getting points for a clean sheet.


In [49]:
# Clean the data
team_defence_df.drop(['# Pl',	'90s',	'SoT/90',	'G-xG', 'SoT', 'FK', 'PK', 'npxG'], axis=1, inplace=True)

In [50]:
# Switch to numeric
team_defence_df[['Gls',	'Sh', 'SoT%',	'Sh/90',	'G/Sh',	'G/SoT',	'Dist',	'PKatt',	'xG',	'npxG/Sh',	'np:G-xG']] = team_defence_df[['Gls',	'Sh', 'SoT%',	'Sh/90',	'G/Sh',	'G/SoT',	'Dist',	'PKatt',	'xG',	'npxG/Sh',	'np:G-xG']].apply(pd.to_numeric, errors='coerce')


## First let's plot teams that concede a lot of shots and are overachieving defensively (They are letting in less goals than their xG is suggesting

In [51]:
fig = px.scatter(team_defence_df,
                 x='Sh',
                 y='np:G-xG',
                 size='Gls',
                 color='npxG/Sh',
                 text='Squad',
                 title= "Teams that concede a lot of shots and are overachieving in terms of xG against them",
                 )

fig.update_layout(xaxis_title='Number of shots conceded per game',
                  yaxis_title='Goals conceded minus the expected goals'
                  )

fig.update_layout(shapes=[
    dict(
        type='line',
        x0=full['Sh'].min(),
        x1=full['Sh'].max(),
        y0=0,
        y1=0,
        line=dict(color='black', width=2)
    )
])

fig.show()

Teams above the 0 line have conceded less goals than their data suggests. for teams that play a high line with lots of risk for a better offensive output like Liverpool and Tottenham(although their overachievement is a little worrying) this is ok, especially as they haven't conceded that many shots against them. Alison and Vicario are probably not a smart option due to this.

The numbers of West Ham and Man United are a concern. I guess there is something in Areola, not just his price, the average xG of a chance West Ham give to their opponents is rather low. But there is Turner from Forest who is similarly priced but should have a better output in the long term. They should have some stability defensively under Nuno and he is a player you should look out for.

Arsenal and City are the obvious choice, although City do give away some high quality chances, but their goalkeepers are too expensive and you probably will not want to take up a spot from these teams on your goalkeeper.

Brentford look like a good alternative option. The quality of chance they give away is among the worst in the league and they don't give away too many shots and they are underachiving as their XG is concerned. They should improve in the second half of the season.

In [52]:
# Clean the data
keeper_df.drop(['Rk', 'Nation', 'Pos', 'Age', 'Born', 'MP', 'Min', 'D', 'W', 'L', 'CS%', 'PKatt', 'PKA', 'PKsv', 'PKm', 'Matches'], axis=1, inplace=True)

In [53]:
# Switch numeric columns from string to numeric
keeper_df[['Starts',	'90s',	'GA',	'GA90',	'SoTA',	'Saves',	'Save%',	'CS',	'Save%.1']] = keeper_df[['Starts',	'90s',	'GA',	'GA90',	'SoTA',	'Saves',	'Save%',	'CS',	'Save%.1']].apply(pd.to_numeric, errors='coerce')

# Drop player column which contains player due to scraping error. Later to adjust in the scrape code
keeper_df.drop(keeper_df[keeper_df['Player'] == 'Player'].index, inplace=True)

keeper = keeper_df[keeper_df.Starts > (keeper_df.Starts.max() * 0.6)]

keeper.sample(5)

,Player,Squad,Starts,90s,GA,GA90,SoTA,Saves,Save%,CS,Save%.1
4,Ederson,Manchester City,18.00,18.00,20.00,1.11,51.00,32.00,64.70,5.00,0.00
26,José Sá,Wolves,18.00,17.20,28.00,1.62,79.00,52.00,70.90,2.00,0.00
20,Jordan Pickford,Everton,20.00,20.00,28.00,1.40,79.00,53.00,70.90,6.00,0.00
13,Emiliano Martínez,Aston Villa,19.00,19.00,26.00,1.37,80.00,56.00,67.50,4.00,NaN
30,James Trafford,Burnley,20.00,20.00,41.00,2.05,120.00,81.00,68.30,2.00,0.00


# Compare the number of clean sheets with the number of shots against them


In [54]:
fig = px.scatter(keeper,
                 x='CS',
                 y='SoTA',
                 size='GA',
                 color='Saves',
                 text='Player',
                 title= "How many shots on target a goalkeeper has face and how many clean sheets he has",
                 )

fig.update_layout(xaxis_title='Clean Sheets',
                  yaxis_title='Number of shots on target faced'
                  )

fig.show()

Best goalkeeper has only 6 clean sheets from 20 games pleyed. That seems like a rather low number. Another reason to not overspend on goalkeepers.

We see that Onana defies his stats by far, but is that sustainable. If you've watched United this season, you will se that they have been lucky a few times.

Pickford jumps at us as a surprise. With Dyches's reputation and Everton becomming more and more his kind of team it's an option to consider. But another one is Sam Johnstone with 5 clean sheets and very low number of shots on target faced. He should be returning from injury soon and is worth thinking about.

Just a quick note, another player who isn't on the list is Dubravka who hasn't played more than 60% of fixtures, but since Pope has had 5 clean sheets and is ranked very good on the chart. Dubravka with his price can be a very good option.

# Filter out the best defensive teams to get see from which teams to choose defenders

I will filter out teams:

*   that have Conceded goals below the league average
*   that have faced the amount of shots that are below the league average
*   that have expected goals against below the league average

I have tried runnig this for all three cases but I only got 6 teams which is not enough so i widened the serach to two out of three and got nine teams which gives me enough defenders to work with.

In [55]:
best_defence = team_defence_df[((team_defence_df.Gls < team_defence_df.Gls.mean()) & (team_defence_df.Sh < team_defence_df.Sh.mean()))
                                | ((team_defence_df.Gls < team_defence_df.Gls.mean()) & (team_defence_df.xG < team_defence_df.xG.mean()))
                                | ((team_defence_df.xG < team_defence_df.xG.mean()) & (team_defence_df.Sh < team_defence_df.Sh.mean()))
                                ]

best_defence.sample(5)

,Squad,Gls,Sh,SoT%,Sh/90,G/Sh,G/SoT,Dist,PKatt,xG,npxG/Sh,np:G-xG
0,vs Arsenal,20,170,27.60,8.50,0.11,0.40,17.70,2,16.30,0.09,4.20
4,vs Brighton,31,241,31.50,12.05,0.11,0.34,16.20,5,29.60,0.11,0.40
6,vs Chelsea,31,248,37.90,12.40,0.12,0.31,16.90,3,28.80,0.11,2.60
10,vs Liverpool,17,225,29.80,11.25,0.07,0.24,17.20,1,22.60,0.10,-5.80
3,vs Brentford,31,248,32.70,13.05,0.12,0.37,17.80,1,25.20,0.10,5.50


In [56]:
# Change the name of Squad column to exclude vs to be able to filter out the defensive players in the next step
best_defence['Squad'] = best_defence['Squad'].apply(lambda x: x.split(' ')[1] if len(x.split(' ')) < 3 else ' '.join(x.split(' ')[1:]))

<ipython-input-56-e9ac4d6f79ed>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
# Filter out the chosen defensive teams and just the defenders from the list
def_shooting = shooting_df[(shooting_df['Squad'].isin(best_defence.Squad.values)) & ((shooting_df.Pos == 'DF') | (shooting_df['Pos'].str.split(',')[0] == 'DF'))]

In [58]:
# Merge def shooting with creativity to get offensive output of defenders
deff_attack = def_shooting.merge(creativity_df, on=['Player', 'Pos', 'Squad', '90s'], how='left')

deff_attack.sample(5)

,Player,Pos,Squad,90s,Gls,Sh,Sh/90,G/Sh,PKatt,xG,npxG,Ast,xA,KP,PPA,CrsPA,FK,CK
17,Pervis Estupiñán,DF,Brighton,7.00,2.00,9.00,1.29,0.22,0.00,0.90,0.90,3.00,1.50,15.00,19.00,6.00,12.00,0.00
9,Nathan Collins,DF,Brentford,13.30,1.00,7.00,0.53,0.14,0.00,1.20,1.20,1.00,0.60,3.00,2.00,0.00,21.00,0.00
21,Marc Guéhi,DF,Crystal Palace,19.00,0.00,5.00,0.26,0.00,0.00,0.60,0.60,1.00,0.90,4.00,3.00,0.00,44.00,0.00
58,Joel Ward,DF,Crystal Palace,14.80,0.00,9.00,0.61,0.00,0.00,0.70,0.70,1.00,0.20,3.00,7.00,3.00,15.00,0.00
63,Oleksandr Zinchenko,DF,Arsenal,13.40,1.00,13.00,0.97,0.08,0.00,0.40,0.40,1.00,2.20,19.00,31.00,2.00,35.00,0.00


In [59]:
# Filter out players that have played at least half the minutes
deff_attack = deff_attack[deff_attack['90s'] > deff_attack['90s'].max() / 2]

In [60]:
# Add the column for getting the ball into the penalty area and taking dead ball situations
deff_attack['PA'] = deff_attack.PPA + deff_attack.CrsPA
deff_attack['DB'] = deff_attack.FK + deff_attack.CK

deff_attack.drop(['G/Sh', 'PKatt', 'PPA', 'CrsPA', 'FK', 'CK'], axis=1, inplace=True)

deff_attack.sample(5)

,Player,Pos,Squad,90s,Gls,Sh,Sh/90,xG,npxG,Ast,xA,KP,PA,DB
21,Marc Guéhi,DF,Crystal Palace,19.00,0.00,5.00,0.26,0.60,0.60,1.00,0.90,4.00,3.00,44.00
2,Trent Alexander-Arnold,DF,Liverpool,16.60,2.00,29.00,1.74,1.80,1.50,3.00,4.20,49.00,46.00,114.00
49,James Tarkowski,DF,Everton,20.00,1.00,14.00,0.70,1.30,1.30,1.00,0.80,12.00,16.00,24.00
10,Levi Colwill,DF,Chelsea,16.80,1.00,10.00,0.60,1.20,1.20,1.00,0.40,6.00,4.00,24.00
39,Tyrick Mitchell,DF,Crystal Palace,18.10,1.00,3.00,0.17,0.50,0.50,2.00,1.10,11.00,24.00,9.00


# Let's see which players create a lot of chances

In [61]:
fig = px.scatter(deff_attack,
                 x='PA',
                 y='KP',
                 size='xA',
                 color='90s',
                 text='Player',
                 title= "See which players create the most chances",
                 )

fig.update_layout(xaxis_title='Passes into the penalty area',
                  yaxis_title='Key passes leading to a shot'
                  )

fig.show()

Trent is of course way above everyone. Digne is the second best option in the sense of getting the ball to players in the Penalty area. Zinchenko is interesting, but his game time isn't very consistent. Walker is your safe option. White looks the best option from Arsenal, and thinking outside the box might land you a Crystal Palace defender like Andersen or Mitchel or even perhaps Patterson from Everton if he is fit


In [62]:
# Add columng for total Goals and Assists
deff_attack['GA'] = deff_attack.Gls + deff_attack.Ast
deff_attack['xGA'] = deff_attack.xG + deff_attack.xA

## Let's see what expected numbers tell us for our defenders

In [93]:
fig = px.scatter(deff_attack,
                 x='xG',
                 y='xA',
                 size='Sh',
                 color='90s',
                 text='Player',
                 title= "The expected numbers of players",
                 )

fig.update_layout(xaxis_title='Expected Goals',
                  yaxis_title='Expected Assists'
                  )

fig.show()

Dunk does come out on top with a sursprisingly high xG, but how much can you really expect from a defender in terms of scoring. Might be an interesting option even though his actual numbers are pretty bad. Birghton have had only one clean sheet this season but their data plays a different picture. The focus should really be on creativity.

That is why Zinchenko and Digne are the best options when fit. Appart from that noone really stands out that much that I haven't already mentioned.

Another special mention to Trippier and Porro whose teams data didn't qualify them for this list, but sometimes you have to use commmons sense and look at the actual game. They are definitely near the top of all defenders.